# Getting Started with QuantumEdge Portfolio Optimization

Welcome to QuantumEdge! This notebook will guide you through the basics of quantum-inspired portfolio optimization using our platform.

## What You'll Learn

1. Setting up the QuantumEdge environment
2. Loading market data
3. Basic portfolio optimization using mean-variance
4. Interpreting optimization results
5. Visualizing portfolio allocations

## Prerequisites

- Python 3.8+
- Basic understanding of portfolio theory
- Familiarity with NumPy and Pandas


## 1. Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# QuantumEdge modules
import sys
sys.path.append('../src')

from optimization import MeanVarianceOptimizer, ObjectiveType
from data.yahoo_finance import YahooFinanceProvider
from backtesting import BacktestEngine, BacktestConfig, BuyAndHoldStrategy

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ QuantumEdge environment set up successfully!")

## 2. Loading Market Data

Let's start by loading some real market data for a diversified set of assets.

In [ ]:
# Define our universe of assets
symbols = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'NVDA', 'JPM', 'JNJ']
print(f"Asset Universe: {symbols}")

# Initialize data provider
data_provider = YahooFinanceProvider()

# Define date range (last 2 years)
end_date = datetime.now().date()
start_date = (datetime.now() - timedelta(days=730)).date()

print(f"Data Range: {start_date} to {end_date}")

In [ ]:
# Fetch historical data
print("Fetching historical price data...")

price_data = {}
for symbol in symbols:
    try:
        market_data = await data_provider.get_historical_data(
            symbol=symbol,
            start_date=start_date,
            end_date=end_date,
            frequency='DAILY'
        )
        
        # Extract adjusted close prices
        prices = pd.DataFrame([
            {
                'date': price.timestamp.date(),
                'price': price.adjusted_close or price.close
            }
            for price in market_data.data
        ])
        prices.set_index('date', inplace=True)
        price_data[symbol] = prices['price']
        
    except Exception as e:
        print(f"⚠️  Failed to fetch data for {symbol}: {e}")

# Combine into DataFrame
prices_df = pd.DataFrame(price_data)
prices_df = prices_df.dropna()  # Remove any missing data

print(f"✅ Loaded price data for {len(prices_df.columns)} assets")
print(f"   Data shape: {prices_df.shape}")
print(f"   Date range: {prices_df.index.min()} to {prices_df.index.max()}")

# Display first few rows
print("\nFirst 5 rows of price data:")
prices_df.head()

## 3. Calculate Returns and Risk Metrics

Now let's calculate daily returns and compute the expected returns and covariance matrix needed for optimization.

In [ ]:
# Calculate daily returns
returns_df = prices_df.pct_change().dropna()

print(f"Returns shape: {returns_df.shape}")
print(f"Returns period: {returns_df.index.min()} to {returns_df.index.max()}")

# Calculate annualized expected returns (252 trading days per year)
expected_returns = returns_df.mean() * 252

# Calculate annualized covariance matrix
covariance_matrix = returns_df.cov() * 252

print("\n📊 Expected Annual Returns:")
for symbol, ret in expected_returns.items():
    print(f"   {symbol}: {ret:.2%}")

print("\n📈 Annual Volatilities:")
volatilities = np.sqrt(np.diag(covariance_matrix))
for symbol, vol in zip(expected_returns.index, volatilities):
    print(f"   {symbol}: {vol:.2%}")

## 4. Visualize Risk-Return Characteristics

Let's create a scatter plot to visualize the risk-return profile of our assets.

In [ ]:
# Create risk-return scatter plot
fig, ax = plt.subplots(figsize=(12, 8))

# Plot each asset
for i, symbol in enumerate(expected_returns.index):
    ax.scatter(
        volatilities[i], 
        expected_returns[symbol], 
        s=100, 
        alpha=0.7,
        label=symbol
    )
    
    # Add asset labels
    ax.annotate(
        symbol, 
        (volatilities[i], expected_returns[symbol]),
        xytext=(5, 5), 
        textcoords='offset points',
        fontsize=10,
        fontweight='bold'
    )

ax.set_xlabel('Annual Volatility (%)', fontsize=12)
ax.set_ylabel('Expected Annual Return (%)', fontsize=12)
ax.set_title('Risk-Return Profile of Assets', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Format axes as percentages
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.1%}'))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, p: f'{y:.1%}'))

plt.tight_layout()
plt.show()

# Print correlation matrix
print("\n📊 Correlation Matrix:")
correlation_matrix = returns_df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'label': 'Correlation'})
plt.title('Asset Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Portfolio Optimization with Mean-Variance

Now let's use QuantumEdge's mean-variance optimizer to find optimal portfolio allocations.

In [ ]:
# Initialize the mean-variance optimizer
optimizer = MeanVarianceOptimizer(
    risk_free_rate=0.02,  # 2% risk-free rate
    verbose=False
)

print("🚀 Running portfolio optimization...")

# Convert to numpy arrays for optimization
returns_array = expected_returns.values
cov_matrix = covariance_matrix.values

# Optimize for maximum Sharpe ratio
result = optimizer.optimize_portfolio(
    expected_returns=returns_array,
    covariance_matrix=cov_matrix,
    objective=ObjectiveType.MAXIMIZE_SHARPE
)

if result.success:
    print("✅ Optimization successful!")
    
    # Display results
    print(f"\n📈 Portfolio Metrics:")
    print(f"   Expected Return: {result.expected_return:.2%}")
    print(f"   Volatility: {np.sqrt(result.expected_variance):.2%}")
    print(f"   Sharpe Ratio: {result.sharpe_ratio:.3f}")
    print(f"   Solve Time: {result.solve_time:.3f}s")
    
    # Create weights DataFrame
    optimal_weights = pd.Series(
        result.weights, 
        index=expected_returns.index, 
        name='Weight'
    )
    
    print(f"\n🎯 Optimal Portfolio Weights:")
    for symbol, weight in optimal_weights.items():
        if weight > 0.001:  # Only show meaningful allocations
            print(f"   {symbol}: {weight:.1%}")
    
else:
    print(f"❌ Optimization failed: {result.status}")
    print(f"   Message: {result.message if hasattr(result, 'message') else 'No message'}")

## 6. Visualize Portfolio Allocation

Let's create visualizations to better understand our optimal portfolio.

In [ ]:
if result.success:
    # Create pie chart for portfolio allocation
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # Pie chart - only show allocations > 1%
    significant_weights = optimal_weights[optimal_weights > 0.01]
    other_weight = optimal_weights[optimal_weights <= 0.01].sum()
    
    if other_weight > 0:
        plot_weights = significant_weights.copy()
        plot_weights['Others'] = other_weight
    else:
        plot_weights = significant_weights
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(plot_weights)))
    wedges, texts, autotexts = ax1.pie(
        plot_weights.values, 
        labels=plot_weights.index,
        autopct='%1.1f%%',
        colors=colors,
        startangle=90
    )
    ax1.set_title('Optimal Portfolio Allocation', fontsize=14, fontweight='bold')
    
    # Bar chart for all weights
    optimal_weights.plot(kind='bar', ax=ax2, color='steelblue', alpha=0.7)
    ax2.set_title('Portfolio Weights by Asset', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Assets', fontsize=12)
    ax2.set_ylabel('Weight', fontsize=12)
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, p: f'{y:.1%}'))
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Portfolio contribution analysis
    portfolio_contributions = pd.DataFrame({
        'Weight': optimal_weights,
        'Expected Return': expected_returns,
        'Volatility': volatilities,
        'Return Contribution': optimal_weights * expected_returns,
        'Risk Contribution': optimal_weights * volatilities
    })
    
    # Only show significant positions
    significant_positions = portfolio_contributions[portfolio_contributions['Weight'] > 0.01]
    
    print("\n📊 Portfolio Contribution Analysis:")
    print(significant_positions.round(4))

## 7. Compare Different Optimization Objectives

Let's compare how different objectives affect our portfolio allocation.

In [ ]:
# Define different objectives to compare
objectives = [
    (ObjectiveType.MAXIMIZE_SHARPE, "Max Sharpe"),
    (ObjectiveType.MINIMIZE_VARIANCE, "Min Variance"),
    (ObjectiveType.MAXIMIZE_RETURN, "Max Return"),
    (ObjectiveType.MAXIMIZE_UTILITY, "Max Utility")
]

# Store results for comparison
comparison_results = {}

print("🔄 Comparing different optimization objectives...\n")

for objective, name in objectives:
    result = optimizer.optimize_portfolio(
        expected_returns=returns_array,
        covariance_matrix=cov_matrix,
        objective=objective,
        risk_aversion=1.0  # For utility maximization
    )
    
    if result.success:
        comparison_results[name] = {
            'weights': pd.Series(result.weights, index=expected_returns.index),
            'return': result.expected_return,
            'volatility': np.sqrt(result.expected_variance),
            'sharpe': result.sharpe_ratio
        }
        
        print(f"✅ {name}:")
        print(f"   Return: {result.expected_return:.2%}")
        print(f"   Volatility: {np.sqrt(result.expected_variance):.2%}")
        print(f"   Sharpe: {result.sharpe_ratio:.3f}\n")
    else:
        print(f"❌ {name}: Optimization failed\n")

# Create comparison visualization
if comparison_results:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # Risk-Return scatter
    for name, data in comparison_results.items():
        ax1.scatter(
            data['volatility'], 
            data['return'], 
            s=150, 
            alpha=0.8,
            label=name
        )
        ax1.annotate(
            name, 
            (data['volatility'], data['return']),
            xytext=(5, 5), 
            textcoords='offset points',
            fontsize=10
        )
    
    ax1.set_xlabel('Volatility', fontsize=12)
    ax1.set_ylabel('Expected Return', fontsize=12)
    ax1.set_title('Risk-Return Comparison', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Sharpe ratio comparison
    sharpe_ratios = [data['sharpe'] for data in comparison_results.values()]
    names = list(comparison_results.keys())
    
    bars = ax2.bar(names, sharpe_ratios, color='lightcoral', alpha=0.7)
    ax2.set_ylabel('Sharpe Ratio', fontsize=12)
    ax2.set_title('Sharpe Ratio Comparison', fontsize=14, fontweight='bold')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, sharpe_ratios):
        ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

## 8. Summary and Next Steps

Congratulations! You've successfully completed your first QuantumEdge portfolio optimization.

### What We Accomplished

1. ✅ Set up the QuantumEdge environment
2. ✅ Loaded real market data for 8 major stocks
3. ✅ Calculated expected returns and risk metrics
4. ✅ Visualized asset risk-return characteristics
5. ✅ Performed mean-variance optimization
6. ✅ Compared different optimization objectives
7. ✅ Created portfolio allocation visualizations

### Key Insights

- Mean-variance optimization helps balance risk and return
- Different objectives lead to different portfolio allocations
- The Sharpe ratio maximization typically provides good risk-adjusted returns
- Diversification reduces portfolio risk through correlation effects

### Next Steps

1. **Advanced Objectives**: Explore CVaR, Sortino, and Calmar ratio optimization
2. **Quantum Methods**: Try VQE and QAOA quantum-inspired algorithms
3. **Backtesting**: Test your strategies against historical data
4. **Risk Analysis**: Dive deeper into portfolio risk metrics
5. **Real-time Optimization**: Connect to live market data feeds

Ready to explore more? Check out the other notebooks in this series:
- `02_quantum_vs_classical.ipynb` - Quantum vs Classical comparison
- `03_backtesting_strategies.ipynb` - Historical performance analysis
- `04_advanced_objectives.ipynb` - CVaR, Sortino, and Calmar optimization
- `05_risk_analysis.ipynb` - Deep dive into risk metrics